In [1]:
import lightgbm as lgb

C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\45928\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
lgb.__version__

'3.2.1'

In [1]:
import os
import pandas as pd

In [ ]:
# 2.1.2

In [2]:
path=""

In [3]:
test=pd.read_json(path+"CVPR_2022_NAS_Track2_test.json").T

In [4]:
train_ori=pd.read_json(path+"CVPR_2022_NAS_Track2_train.json").T
#test=pd.read_json(path+"CVPR_2022_NAS_Track2_test.json").T

In [5]:
label_list=['cplfw_rank', 'market1501_rank', 'dukemtmc_rank', 'msmt17_rank',
       'veri_rank', 'vehicleid_rank', 'veriwild_rank', 'sop_rank']
for i in label_list:
    train_ori[i]=train_ori[i].astype(int)

In [12]:
train=train_ori[:500].reset_index(drop=True)
valid=train_ori[400:].reset_index(drop=True)

In [13]:
def get_str(x,s):
    r=""
    for i in range(12):
        r+=x[i*3+s]
    return r

def create_data(data):
    fe_list=[]
    #l:12,k:11,j:10
    data["fe00"]=data["arch"].apply(lambda x:x[0]).apply(lambda x:{"j":0,"k":1,"l":2}[x])
    fe_list.append("fe00")
    #"""
    for i in range(12):
        data["fe%s"%i]=data["arch"].apply(lambda x:int(x[i*3+1:i*3+2]))#.apply(lambda x:{1:12,2:11,3:10,0:0}[x])
        fe_list.append("fe%s"%i)
    for i in range(12):
        data["fe%s"%(i+12)]=data["arch"].apply(lambda x:int(x[i*3+2:i*3+3]))#.apply(lambda x:{1:4,2:3.5,3:3,0:0}[x])
        fe_list.append("fe%s"%(i+12))
    for i in range(12):
        data["fe%s"%(i+24)]=data["arch"].apply(lambda x:int(x[i*3+3:i*3+4]))#.apply(lambda x:{1:4,2:3.5,3:3,0:0}[x])
        fe_list.append("fe%s"%(i+24))
    
    return data,fe_list


In [14]:
train,fe_list=create_data(train)
valid,fe_list=create_data(valid)
test,fe_list=create_data(test)

In [15]:
for l in label_list:
    train[l]=(train[l].rank()-1).astype(int)
    valid[l]=(valid[l].rank()-1).astype(int)

In [16]:
n=0
tag_dic={}
for l in label_list:
    tag_dic[l]=n
    n+=1

In [18]:
import lightgbm as lgb
import xgboost as xgb
import torch
import scipy
import numpy as np

params = {
        'num_leaves': 2,
        'objective': 'regression_l2',
        'max_depth': 10,
        'min_data_in_leaf': 1,
        'learning_rate': 0.8,
        'feature_fraction': 0.99,
        'bagging_fraction': 0.99,
        'bagging_freq': 1,
        'metric': 'mse',
        'num_threads': 32,
        'seed':2018
          }

xgbparams = {'booster': 'gbtree',
          'eval_metric': 'rmse',
          'gamma': 1,
          'min_child_weight': 5,
          'max_depth': 1,
          'lambda': 10,
          'eta': 0.8,
          'tree_method': 'exact',
          'seed': 2022,
          'nthread': 32,
          'silent':True
          }


num_round = {}
early_stopping_rounds = 100

score_list=[]
for l in label_list:

    
    num_round[l]=300
    #"""
    num_round["cplfw_rank"]=50
    num_round["market1501_rank"]=200
    num_round["dukemtmc_rank"]=300
    num_round["msmt17_rank"]=500
    num_round["veri_rank"]=100
    num_round["vehicleid_rank"]=100
    num_round["veriwild_rank"]=250
    num_round["sop_rank"]=100
    #"""
    
    pred_list=[]
    sub_list=[]
    for i in range(20):
        train_temp=train.sample(frac=0.75,random_state=i).reset_index(drop=True)
        train_temp[l]=(train_temp[l].rank()-1).astype(int)
        #train_temp[l]=train_temp[l].apply(lambda x:x+1 if x%2==0 else x-1)
        train_y=train_temp[l]

        #"""
        from scipy.special import erfinv
        mmin=np.min(train_y)+1
        mmax=np.max(train_y)+1
        train_y=np.sqrt(2) * erfinv(2 * (train_y+mmin)/(mmin+mmax)-1)
        #"""
        
        #train_y=(train_y-train_y.mean())/train_y.mean()
        #train_y=train_y*(np.abs(train_y)**0.3)
        
        params["seed"]=i
        if l in []:#label_list:
            train_matrix = xgb.DMatrix(train_temp[fe_list], label=train_y)
            valid_matrix = xgb.DMatrix(valid[fe_list], label=train_y)
            test_matrix = xgb.DMatrix(test[fe_list], label=train_y)
            
            model = xgb.train(xgbparams, train_matrix, num_round[l])

            pred=model.predict(valid_matrix)
            sub=model.predict(test_matrix)
            
            
        else:
            train_matrix = lgb.Dataset(train_temp[fe_list], label=train_y,#weight=1-np.abs(train_temp[l]-train_temp["w"])/train_temp[l].max()
                                      )
            cnt=0
            model = lgb.train(params, train_matrix,num_round[l]#,fobj=myloss
                              #valid_sets=test_matrix,verbose_eval=50,
                              #early_stopping_rounds=early_stopping_rounds
                              )

            #print("\n".join(("%s: %.2f" % x) for x in list(sorted(zip(features, model.feature_importance("gain")), key=lambda x: x[1],reverse=True))[:200]))
            pred=model.predict(valid[fe_list])
            sub=model.predict(test[fe_list])
                
        pred_list.append(pred)
        sub_list.append(sub)
            
    pred=np.mean(np.array(pred_list),axis=0)
    sub=np.mean(np.array(sub_list),axis=0)

    score=scipy.stats.stats.kendalltau(pred, valid[l])[0]
    print(l,score)
    score_list.append(score)
    test[l]=sub
    test[l]=test[l].rank().astype(int)
    
    #break
    
print("score:",np.mean(score_list))

cplfw_rank 0.2840404040404041
market1501_rank 0.8513131313131315
dukemtmc_rank 0.9103030303030304
msmt17_rank 0.9600000000000002
veri_rank 0.9074747474747475
vehicleid_rank 0.6989898989898992
veriwild_rank 0.9365656565656567
sop_rank 0.7907070707070709
score: 0.7924242424242425


In [ ]:
test[label_list+["arch"]].T.to_json(path+"sub_0.7922727272727274.json")